In [33]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pylab as plt

In [34]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
import seaborn as sns
sns.set()

In [37]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,learning_curve, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier

In [38]:
from sklearn.cluster import KMeans

In [40]:
train = pd.read_csv("../../project_data/galaxy_train.csv", index_col=0)
test = pd.read_csv('../../project_data/galaxy_test.csv', index_col=0)
sample_submission = pd.read_csv('../../project_data/sample_submission.csv', index_col=0)

In [41]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [42]:
train.head()

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,type_num
id,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,...,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340,8
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,...,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314,8
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,...,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928,8
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,...,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855,8
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,...,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655,10


In [21]:
y_galaxy_train = train["type_num"]
X_galaxy_train = train.iloc[:,2:-1]

In [26]:
train['type'].value_counts()

QSO                    49680
GALAXY                 37347
SERENDIPITY_BLUE       21760
SPECTROPHOTO_STD       14630
REDDEN_STD             14618
STAR_RED_DWARF         13750
STAR_BHB               13500
SERENDIPITY_FIRST       7132
ROSAT_D                 6580
STAR_CATY_VAR           6506
SERENDIPITY_DISTANT     4654
STAR_CARBON             3257
SERENDIPITY_RED         2562
STAR_WHITE_DWARF        2160
STAR_SUB_DWARF          1154
STAR_BROWN_DWARF         500
SKY                      127
SERENDIPITY_MANUAL        61
STAR_PN                   13
Name: type, dtype: int64

In [27]:
y_galaxy_train = temp_train["type_num"]
X_galaxy_train = temp_train.iloc[:,2:-1]

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_galaxy_train, 
                                                    y_galaxy_train, 
                                                    stratify = y_galaxy_train,
                                                    test_size=0.2, 
                                                    random_state=1234)

In [29]:
kfold = KFold(n_splits=5, random_state= 0, shuffle = True)

In [31]:
rf_clf = RandomForestClassifier(n_jobs=-1, random_state=1234)

In [47]:
tuned_parameters = {
    'n_estimators': [300],
    'max_features' : [5],
    'criterion' : ['entropy'],
    'bootstrap' : [True]
}

In [48]:
n_iter_search = 10
RF_rgs = RandomizedSearchCV(rf_clf, 
                             tuned_parameters, 
                             cv=kfold,
                             n_jobs=-1,
                             random_state=1234,
                             scoring='accuracy')

In [49]:
RF_rgc = RF_rgs.fit(X_train, y_train)

C:\Users\1-17\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


In [50]:
RF_rgc.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=5, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=1234, verbose=0,
                       warm_start=False)

In [51]:
RF_rgc.best_score_

0.9372622918797182

In [52]:
n_iter_search = 10
RF_rgs = GridSearchCV(rf_clf, 
                             tuned_parameters, 
                             cv=kfold,
                             n_jobs=-1,
                             scoring='accuracy')

In [53]:
RF_rgc = RF_rgs.fit(X_train, y_train)

In [54]:
RF_rgc.best_score_

0.9372622918797182

In [ ]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission.csv', index=True)